In [2]:
import sys
sys.path.append('../')

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
%reload_ext autoreload

In [3]:
from model.scripts.packages.detect_voids import get_neightbours, search_voids_bb_neightbours
from model.scripts.packages.yolo_predict import YOLO_Pred
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import cv2
import multiprocessing
from functools import partial
import itertools
import os

ModuleNotFoundError: No module named 'scripts'

In [69]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [70]:
img_path = '/home/cremerf/FinalProject/data/images/test/test_7.jpg'

In [71]:
image = cv2.imread(img_path)
h_image, w_image = image.shape[0:2] # limits of the image
df_predictions = yolo.predictions(image=image)

# Get neightbours from 3 ways (right / left / up)
pool = multiprocessing.Pool()
neightbour = 'left'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_left = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

# clean up empty positions
dict_of_neightbours_left = list(filter(None, dict_of_neightbours_left))

pool = multiprocessing.Pool()
neightbour = 'right'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_right = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_right = list(filter(None, dict_of_neightbours_right))

'''
pool = multiprocessing.Pool()
neightbour = 'up'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_up = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_up = list(filter(None, dict_of_neightbours_up))
'''

# Merged 3 separated list of dicts(left/right/high) into 1 list
dicts_neightbours = [dict_of_neightbours_left, dict_of_neightbours_right]
list_neightbours = list(itertools.chain.from_iterable(dicts_neightbours))

# Get void neightbours 
list_of_voids = search_voids_bb_neightbours(df_predictions=df_predictions, list_of_dicts=list_neightbours, h_image=h_image, w_image=w_image, img_path= img_path)

# Create dataframe with data(X_center/Y_center/Label) of voids
df_voids = pd.DataFrame(list_of_voids, columns=['Neightbour','Label','x1','y1','x2','y2', 'Width','Height'])

In [72]:
df_predictions

,X_center,Y_center,Width,Height
572,1130.014352,1566.114157,119.923841,347.048751
1059,1073.070552,1188.697504,125.808713,359.978320
581,1603.940973,1564.580640,83.080911,310.984321
585,1751.918793,1564.916821,83.728849,308.152010
1203,759.620471,2645.523212,138.472683,139.737216
...,...,...,...,...
1145,607.823254,2371.570038,127.150735,75.156188
712,2275.615549,2464.040222,214.330290,79.787421
310,2179.846271,1621.568875,53.256595,146.695816
600,856.638524,1615.428113,107.553291,300.801130


In [73]:
df_voids

,Neightbour,Label,x1,y1,x2,y2,Width,Height
0,92,Void_1,1960.436034,2224.043330,2068.031620,2368.230877,107.595586,144.187547
1,1145,Void_2,417.097152,2333.991944,544.247887,2409.148132,127.150735,75.156188
2,943,Void_3,1926.983928,2240.429894,2059.756581,2385.724574,132.772653,145.294680


In [48]:
df_voids

,Neightbour,Label,x1,y1,x2,y2,Width,Height
0,203,203 void #1,239.126488,1587.569778,346.302681,1724.415817,107.176193,136.846039
1,170,170 void #1,522.468880,1284.883480,606.483604,1410.608707,84.014725,125.725227


In [49]:
for i in df_voids.index:
    x1 = int(df_voids.loc[i][2]) 
    y1 = int(df_voids.loc[i][3]) 
    x2 = int(df_voids.loc[i][4])  
    y2 = int(df_voids.loc[i][5])
    print(df_voids.loc[i][1])

203 void #1
170 void #1


In [63]:
#image_path_bb = os.path.join(folder_path_final, filename)

img_path = '/home/cremerf/FinalProject/scripts/test_neightbours2.jpg'
# load the image
image = cv2.imread(img_path)

# loop over indexes and for each index plot the rectangles
#for idx in df_voids.index:

# get the coordinates for each index/rectangle
for i in df_voids.index:
    x1 = int(df_voids.loc[i][2]) 
    y1 = int(df_voids.loc[i][3]) 
    x2 = int(df_voids.loc[i][4])  
    y2 = int(df_voids.loc[i][5])


    # # Window name in which image is displayed
    window_name = 'Object'
    # represents the top left corner of rectangle
    start_point=(x1, y1)

    # represents the top right corner of rectangle
    end_point=(x2,y2)

    # # Blue color in BGR
    color = (0, 0, 255)

    # # Line thickness of 2 px
    thickness = 5

    # plot the rectangle over the image
    cv2.putText(image,str(df_voids.loc[i][1]),(x1,y1-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)
    image = cv2.rectangle(image, start_point, end_point, color, thickness)

# save the img
cv2.imwrite(filename='test_voids5.jpg', img=image)

True

In [ ]:
9	424	424 void #1	785.095965	1608.560190

In [17]:
def image_mean(x, y, w, h, img_path):
    
    image = cv2.imread(img_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    roi = np.mean(gray[y:y + h, x:x + w])

    return roi

In [42]:
df_voids.loc[6]

Neightbour            661
Label         661 void #1
x1                1658.43
y1                1996.02
x2                1831.77
y2                2245.85
Width              173.34
Height            249.831
Name: 6, dtype: object

In [43]:
roi = image_mean(x=1658,y=1996,w=173,h=249,img_path=img_path)

In [44]:
roi

38.753441511711586

In [45]:
df_predictions.loc[661]

X_center    1918.438461
Y_center    2120.930679
Width        173.339578
Height       249.830856
Name: 661, dtype: float64

In [46]:
roi = image_mean(x=int((1918-173/2)),y=int((2120-249/2)),w=173,h=249,img_path=img_path)

In [47]:
roi

91.66453095619472

In [ ]:
roi = image_mean(x=int((2083-145/2)),y=int((2518-320/2)),w=145,h=320,img_path=img_path)